In [1]:
# Install necessary packages
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 466.4 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of embedchain to determine w

In [1]:
import os
import warnings
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from langchain_openai import ChatOpenAI

# Warning control
warnings.filterwarnings('ignore')

# API Key Configuration
# Replace with your actual keys or use a secure method to load them
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_KEY"
os.environ["SERPER_API_KEY"] = "YOUR_SERPER_KEY" # Get from serper.dev

# Model Configuration
# Using gpt-4o-mini or gpt-4-turbo is recommended for complex synthesis,
# but gpt-3.5-turbo works for basic testing.
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:937: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warnings.warn(


In [2]:
# Initialize Tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [3]:
#Agent 1
tech_scout = Agent(
    role="Tech Scout",
    goal="Analyze the target company's website to understand their product, target audience, and monetization model.",
    backstory="You are an expert product researcher. Your job is to visit company landing pages "
              "and translate marketing jargon into clear, factual product descriptions. "
              "You focus on the 'What' and the 'How'.",
    tools=[scrape_tool, search_tool],
    verbose=True,
    allow_delegation=False
)

In [4]:
#Agent 2
market_agent = Agent(
    role="Market Agent",
    goal="Identify the target company's top 3 competitors and evaluate the competitive landscape.",
    backstory="You are a cynical industry analyst. You do not believe the hype. "
              "Your job is to find who else is doing this and why the target company might fail "
              "against incumbents. You look for the 'Moat'.",
    tools=[search_tool],
    verbose=True,
    allow_delegation=False
)

In [5]:
#Agent 3
brand_sentiment_agent = Agent(
    role="Brand Sentiment Agent",
    goal="Gauge real user sentiment from social media (Reddit, Twitter) and review sites.",
    backstory="You are a digital anthropologist. You ignore the press releases and look at "
              "what real people say in comment sections, subreddits, and forums. "
              "You are looking for red flags, churn signals, or fanatical user love.",
    tools=[search_tool],
    verbose=True,
    allow_delegation=False
)

In [6]:
#Agent 4
vc_partner = Agent(
    role="Lead Investor",
    goal="Synthesize reports from the Tech Scout, Market Agent, and Brand Sentiment Agent into a cohesive Investment Memo.",
    backstory="You are a seasoned Venture Capitalist. You take raw data and turn it into "
              "decisive investment memos. You are concise, critical, and focused on ROI. "
              "You must provide a final verdict (Score 1-10) and a recommendation.",
    verbose=True,
    allow_delegation=False
)

In [9]:
# # Task 1: Product Analysis
product_analysis_task = Task(
    description=(
        "Analyze the startup: {company_name} at this URL: {company_url}. "
        "1. Scrape the website to identify the core product offering. "
        "2. Identify the target customer (B2B/B2C/Enterprise). "
        "3. Find the pricing model if available. "
        "4. Summarize the Unique Selling Proposition (USP)."
    ),
    expected_output="A clear, concise report on what the company does, who it sells to, and how it makes money.",
    agent=tech_scout
)

In [10]:
# Task 2: Competitor Analysis
competitor_analysis_task = Task(
    description=(
        "Based on the product analysis of {company_name}, search for direct competitors. "
        "1. Identify the top 3 rivals. "
        "2. Compare their key features briefly. "
        "3. Look for any 'moat' or unfair advantage {company_name} claims to have."
    ),
    expected_output="A table or list of top 3 competitors with a brief comparison of strengths/weaknesses.",
    agent=market_agent
)

In [11]:
# Task 3: Sentiment Analysis
sentiment_analysis_task = Task(
    description=(
        "Search for public sentiment regarding {company_name} and its competitors. "
        "Look specifically on Reddit, Twitter/X, and G2/ProductHunt. "
        "Identify: "
        "1. Common complaints (bugs, pricing, support). "
        "2. What users love the most. "
        "3. Overall sentiment score (Positive/Neutral/Negative)."
    ),
    expected_output="A summary of market sentiment, highlighting specific quotes or recurring themes from users.",
    agent=brand_sentiment_agent
)

In [12]:
# Task 4: The Investment Memo
investment_memo_task = Task(
    description=(
        "Review the reports from the Tech Scout Agent, Market Analysis Agent, and Brand Sentiment Detector Agent. "
        "Draft a professional Investment Memo for {company_name}. "
        "The Memo must include: "
        "1. Executive Summary. "
        "2. Product & Business Model. "
        "3. Competitive Landscape (Risk Analysis). "
        "4. Market Sentiment (User Feedback). "
        "5. Final Verdict: Score (1-10) and Recommendation (Book Meeting / Pass)."
    ),
    expected_output="A professional Markdown formatted Investment Memo.",
    agent=vc_partner,
    context=[product_analysis_task, competitor_analysis_task, sentiment_analysis_task] # Critical for passing data
)

In [13]:
venture_squad = Crew(
    agents=[tech_scout, market_agent, brand_sentiment_agent, vc_partner],
    tasks=[product_analysis_task, competitor_analysis_task, sentiment_analysis_task, investment_memo_task],
    process=Process.sequential,
    verbose=True
)

# Input Data
# You can change these inputs to analyze different startups
inputs = {
    'company_name': 'Perplexity AI',
    'company_url': 'https://www.perplexity.ai',
    'sector': 'Artificial Intelligence / Search'
}

In [14]:
result = venture_squad.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Tech Scout
 [INFO]: == Starting Task: Analyze the startup: Perplexity AI at this URL: https://www.perplexity.ai. 1. Scrape the website to identify the core product offering. 2. Identify the target customer (B2B/B2C/Enterprise). 3. Find the pricing model if available. 4. Summarize the Unique Selling Proposition (USP).


> Entering new CrewAgentExecutor chain...
I need to analyze the Perplexity AI website to gather information about their product offering, target audience, pricing model, and unique selling proposition. The first step is to read the content of their website.

Action: Read website content  
Action Input: {"website_url": "https://www.perplexity.ai"}   

Just a moment...Enable JavaScript and cookies to continue

Thought: It seems that I'm unable to access the website content directly due to JavaScript and cookie restrictions. I should search for information about Perplexity AI using the internet to gather details about their product offering, targ

# Investment Memo: Perplexity AI

## Executive Summary
Perplexity AI is an AI-powered search engine that leverages natural language processing and real-time web data to provide direct, conversational answers to user queries. Its business model combines a freemium approach with premium subscriptions and API access, targeting both individual users and enterprises. Despite the innovative technology, user feedback reveals a mixed sentiment, highlighting both the utility and challenges faced by the platform. With significant growth potential in the AI search market, a careful assessment of risks and competitive positioning is necessary for informed investment decision-making.

## Product & Business Model
### Core Offerings:
1. **AI-Powered Search Engine**: Utilizes large language models (LLMs) to interpret queries and provide contextual answers.
2. **Perplexity Shopping**: Integrates product discovery and recommendations with real-time data for pricing and availability.
3. **Subscription Plans**: 
   - **Free Plan**: Unlimited questions and ad-free experience.
   - **Perplexity Pro**: $20/month for enhanced features.
   - **Perplexity Max**: $200/month for unlimited access to advanced models.

### Revenue Streams:
- **Freemium Model**: Basic services for free, premium features for a fee.
- **API Access**: Businesses can integrate Perplexity's services.
- **Enterprise Partnerships**: Custom solutions for large businesses.
- **Future Advertising**: Potential for ethical ad placements down the line.

## Competitive Landscape (Risk Analysis)
### Key Competitors:
| **Competitor**      | **Strengths**                                                                                     | **Weaknesses**                                                                                  | **USP**                                          |
|----------------------|--------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------|-------------------------------------------------|
| **ChatGPT**          | Strong conversational AI, fast responses, rich plugins                                           | Limited in real-time data retrieval, lacks citations                                          | Versatile creative applications                  |
| **Google Gemini**    | Deep integration with Google services, strong in multimodal tasks                               | Slower in conversational contexts, limited outside Google ecosystem                           | Extensive data leverage for research tasks       |
| **You.com**          | Customizable user experience, privacy-focused                                                    | Weaker AI capabilities, lower brand recognition                                               | Emphasizes user privacy and customization        |

### Risk Assessment:
- **User Experience Issues**: Complaints about model quality and support may deter users.
- **Market Competition**: Strong competitors with established user bases and resources could challenge Perplexity's growth.
- **Scalability Concerns**: The freemium model may not convert enough users to premium subscriptions to sustain expansion.

## Market Sentiment (User Feedback)
User feedback reflects a mixed sentiment:
- **Positive Aspects**: High convenience for quick, accurate answers and well-cited responses.
- **Negative Aspects**: Complaints about answer quality, transparency, and user experience. 

### Quotes from Users:
- "I've found Perplexity to be incredibly convenient for obtaining direct answers in one go."
- "Perplexity is essentially SCAMMING subscribers by marketing their model as 'Sonnet 4.5 Thinking.'"
- "The output quality is terrible."
- "I’m amazed by the focus and technical depth of its answers."

Overall, user sentiment indicates a blend of appreciation and frustration, which could impact retention rates.

## Final Verdict
### Score: 6/10
### Recommendation: Pass
While Perplexity AI presents an innovative solution in the search engine market, the mixed user sentiment and competitive pressures warrant caution. The potential for growth exists, particularly if user experience improvements are prioritized. However, current risks and uncertainties suggest that a meeting for further exploration may not be necessary at this time. 

---

This memo synthesizes insights from the Tech Scout, Market Skeptic, and Culture Spy reports, providing a comprehensive view of Perplexity AI's position in the market and the potential return on investment.

In [ ]:
from IPython.display import Markdown
Markdown(result)